In [1]:
import pandas as pd
import numpy as np
import string
import streamlit as st
import mysql.connector

In [2]:
merchant = pd.read_csv(r'dataset/merchant_demo.csv')
merchant

,m_id,m_tag,m_name,m_province,m_street,comsuption_pp,flavor_rank,packaging_rank,number_comments,number_discount
0,1096261612256400,1,周记汕头牛肉店（牛肉粿条）,广东,深圳市龙岗区园山街道荷坳路57号综合市场B04,32,4.5,4.5,1102,3
1,1023212808498940,1,八合里牛肉火锅（天光城店）,广东,深圳市龙岗区龙城街道爱联社区龙岗大道1099号龙岗天光城2-L-003、004,31,4.5,4.6,328,2
2,1118062866235140,1,港湾火锅到家,广东,深圳市龙岗区龙城街道盛平社区盛丰路,67,4.8,4.9,222,4
3,1094938762285890,1,杨国福麻辣烫（昌盛百货店）,广东,深圳市龙岗区龙城街道爱联社区大道2001号浩天昌盛百货1楼B1商铺,26,4.6,4.7,7368,4
4,862525197024773,1,容川火·毛肚·肥肠煲(龙岗总店),广东,深圳市龙岗区龙城街道爱联社区锦苑路6号104,45,4.5,4.7,2564,5


In [19]:
merchant['flavor_rank'][0]

4.5

In [3]:
comsuption_pp = merchant['comsuption_pp'].tolist()
comsuption_pp

[32, 31, 67, 26, 45]

In [11]:
merchant.length()

AttributeError: 'DataFrame' object has no attribute 'length'

In [6]:
# calculate the consumption level deviation
def consumption_level_deviation(merchant):
    Total = 0
    Count = 0
    comsuption_pp = merchant['comsuption_pp'].tolist()
    for num in comsuption_pp:
        Count += 1
        Total += num
    Average = Total / Count
    return Average
consumption_level_deviation(merchant)


40.2

In [31]:
a = [1,2,3,4]
a.append(1.1)
a

4


[1, 2, 3, 4, 1.1]

In [43]:
def Bayesranking(merchant):
    rank = []
    N = 0 #number of all the ratings
    R = 0 #average of all the rating
    Count = 0
    number_comments = merchant['number_comments'].tolist()
    for num in number_comments:
        N += num
    flavor_rank = merchant['flavor_rank'].tolist()
    for num in flavor_rank:
        R += num
        Count += 1
    R /= Count
    len = merchant.shape[0]
    for i in range(len):
        f_rank = merchant['flavor_rank'][i]
        n_i = merchant['number_comments'][i]
        numerator = N*R + f_rank*n_i
        denominator = N + n_i
        rank_i = numerator / denominator
        print(rank_i)
        rank.append(rank_i)
    # print(rank[0])
    list_len = len(rank)
    for j in range(list_len):
        print(rank[j])
Bayesranking(merchant)


4.5730506069683114
4.5777971793149765
4.58413687955277
4.587775432672013
4.565501837715578


TypeError: 'int' object is not callable

In [ ]:
'''CREATE TABLE merchant ---商家
(M_ID   INT,
M_NAME VARCHAR(20),
M_PROVINCE VARCHAR(25),
M_STREET_ADDRESS VARCHAR(40),
COMSUPTION_PP INT, ---人均消费
FLAVOR_RANK DEC(1,1), ---口味评分
PACKAGING_RANK DEC(1,1), ---包装评分
NUMBER_COMMENTS INT,---评论数
NUMBER_DISCOUNT INT,---套餐数量
PRIMARY KEY(M_ID));'''
# 用户输入一个关键词，通过关键词的检索和content－base算法返回优先队列
